In [1]:
import pandas as pd
import numpy as np

In [2]:
## Definimos path

pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [3]:
## Cargo y ordeno los indicadores

ratio_actividad = pd.read_csv(pathdata + 'data_output/indicadores/01_ratio_actividad.csv', index_col=0)
empleo_asalariado_con_descuento = pd.read_csv(pathdata + 'data_output/indicadores/02_empleo_asalariado_con_descuento.csv', index_col=0)
horas_remuneradas_media = pd.read_csv(pathdata + 'data_output/indicadores/03_horas_promedio_ocup_ppal.csv', index_col=0)
ingreso_salarial_media = pd.read_csv(pathdata + 'data_output/indicadores/04_ingreso_salarial.csv', index_col=0)
ratio_minutos_promedio_no_pago = pd.read_csv(pathdata + 'data_output/indicadores/05_tiempo_no_remunerado.csv', index_col=0)
inactivos_con_ingreso = pd.read_csv(pathdata + 'data_output/indicadores/06_dependencia_economica.csv', index_col=0)
hogares_jefatura = pd.read_csv(pathdata + 'data_output/indicadores/07_hogares_no-pobres_jefatura.csv', index_col=0)

ratio_actividad = ratio_actividad.sort_index()
empleo_asalariado_con_descuento = empleo_asalariado_con_descuento.sort_index()
horas_remuneradas_media = horas_remuneradas_media.sort_index()
ingreso_salarial_media = ingreso_salarial_media.sort_index()
ratio_minutos_promedio_no_pago = ratio_minutos_promedio_no_pago.sort_index()
inactivos_con_ingreso = inactivos_con_ingreso.sort_index()
hogares_jefatura = hogares_jefatura.sort_index()


In [4]:
## Chequeo que todos los indicadores tengan ordenados los indices

variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]

indices_ordenados = True

for i in range(len(variables) - 1):
    if not variables[i].index.equals(variables[i + 1].index):
        indices_ordenados = False
        break

print("Los índices están ordenados de la misma manera:", indices_ordenados)

Los índices están ordenados de la misma manera: True


In [5]:
## Trunco y calculo el ranking de los indicadores

variables = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, hogares_jefatura]
columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

for columna, variable in zip(columnas,variables):
    variable[columna] = np.where(variable[columna] > 100, 100, variable[columna])
    variable['Ranking'] = variable[columna].rank(ascending=False, method='first')


In [6]:
## Agrego componentes e indice

componente_insercion = (ratio_actividad['Actividad'] + \
                        empleo_asalariado_con_descuento['Formalidad'] + \
                        horas_remuneradas_media['Horas promedio remuneradas'] + \
                        ingreso_salarial_media['Ingreso laboral promedio'])/4

componente_insercion = componente_insercion.to_frame()
componente_insercion.columns = ['Insercion laboral']
componente_insercion['Ranking'] = componente_insercion['Insercion laboral'].rank(ascending=False, method='first')

componente_autonomia_economica = (ratio_minutos_promedio_no_pago['Tiempo TNR'] + \
                                inactivos_con_ingreso['Dependencia'] + \
                                hogares_jefatura['No Pobreza'])/3

componente_autonomia_economica = componente_autonomia_economica.to_frame()
componente_autonomia_economica.columns = ['Autonomia economica']
componente_autonomia_economica['Ranking'] = componente_autonomia_economica['Autonomia economica'].rank(ascending=False, method='first')

indice_mti = np.sqrt(componente_autonomia_economica['Autonomia economica'] * componente_insercion['Insercion laboral']).to_frame()
indice_mti.columns = ['Indice MTeI']
indice_mti['Ranking'] = indice_mti['Indice MTeI'].rank(ascending=False, method='first')

In [7]:
## Genero tabla resumen de medias, desvio, maximo, minimo y rango para cada variable

labels_variables_todas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio',
                  'Tiempo TNR', 'Dependencia', 'No Pobreza',
                  'Insercion laboral', 'Autonomia economica', 'Indice MTeI']

columnas = ['Media', 'Desvio estandar', 'Minimo', 'Maximo', 'Rango']

variables_todas = [ratio_actividad, empleo_asalariado_con_descuento, horas_remuneradas_media, \
                    ingreso_salarial_media, ratio_minutos_promedio_no_pago, inactivos_con_ingreso, \
                        hogares_jefatura, componente_insercion, componente_autonomia_economica, indice_mti]

df_tabla_resumen = pd.DataFrame(index=labels_variables_todas, columns=columnas)

for i, idf in enumerate(variables_todas):
    icolumna = labels_variables_todas[i]
    imedia = idf[icolumna].mean()
    istd = idf[icolumna].std()
    imin = idf[icolumna].min() 
    imax = idf[icolumna].max()
    irango = imax - imin
    df_tabla_resumen.iloc[i,0] = imedia
    df_tabla_resumen.iloc[i,1] = istd
    df_tabla_resumen.iloc[i,2] = imin
    df_tabla_resumen.iloc[i,3] = imax
    df_tabla_resumen.iloc[i,4] = irango

df_tabla_resumen = df_tabla_resumen.astype('float').round(2)

In [8]:
## Genero tabla output con todos los indicadores, componentes e indice general.

columnas = ['Actividad', 'Formalidad', 'Horas promedio remuneradas', 'Ingreso laboral promedio', 'Tiempo TNR', 'Dependencia', 'No Pobreza']

provincias = indice_mti.index.to_list()
provincia_repetidos = [value for value in provincias for _ in range(7)]

componentes = ['Insercion laboral'] * 4 + ['Autonomia Economica'] * 3
componentes_repetidos = componentes*24

indicadores = ['Actividad', 'Formalidad', 'Horas trabajadas', 'Ingreso salarial',
                  'Tiempo trabajo no remunerado', 'Dependencia economica', 'No-Pobreza']

data = {
    'Provincia': provincia_repetidos,
    'Componente': componentes_repetidos,
    'Indicador': indicadores*24
}

df = pd.DataFrame(data)

multiindex = pd.MultiIndex.from_frame(df[['Provincia', 'Componente', 'Indicador']])
df.set_index(multiindex, inplace=True)
df.drop(columns=['Provincia', 'Componente', 'Indicador'], inplace=True)

df[['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador', 'Componentes', 'Ranking componente', 'Indice', 'Ranking Indice']] = pd.NaT

for indicador, columna, variable in zip(indicadores, columnas, variables):
    if indicador == 'Tiempo trabajo no remunerado':
        values = variable[['Mujer', 'Varon', columna, 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'Ranking indicador']] = values

    else:    
        values = variable[['Mujer', 'Varon', columna, 'N_pob_v', 'N_pob_m', 'N_pob_tot', 'N_v', 'N_m', 'ER', 'ME', 'Ranking']].values
        df.loc[df.index.get_level_values('Indicador') == indicador, ['Mujer', 'Varon', 'Ratio', 'N pob varon', 'N pob mujer', 'N pob tot', 'N varon', 'N mujer', 'CV', 'Error', 'Ranking indicador']] = values

values = componente_insercion.values
df.loc[df.index.get_level_values('Indicador') == 'Actividad', ['Componentes', 'Ranking componente']] = values

values = componente_autonomia_economica.values
df.loc[df.index.get_level_values('Indicador') == 'Tiempo trabajo no remunerado', ['Componentes', 'Ranking componente']] = values

values = indice_mti.values
df.loc[df.index.get_level_values('Indicador') == 'Actividad', ['Indice', 'Ranking Indice']] = values

In [17]:
## Exporto archivos .csv

df.to_csv(pathdata + 'data_output/tablas/01_indicadores_principales.csv')
componente_insercion.to_csv(pathdata + 'data_output/componentes_e_indice/01_insercion_laboral.csv')
componente_autonomia_economica.to_csv(pathdata + 'data_output/componentes_e_indice/02_insercion_autonomia_economica.csv')
indice_mti.to_csv(pathdata + 'data_output/componentes_e_indice/01_indice_MTeI.csv')

In [22]:
(componente_insercion['Insercion laboral'] - componente_autonomia_economica['Autonomia economica']) / componente_insercion['Insercion laboral'] *100

PROVINCIA
Buenos Aires            6.709881
CABA                    3.724714
Catamarca               4.072318
Chaco                  -1.340546
Chubut                  3.385774
Corrientes              5.341476
Córdoba                 3.690962
Entre Ríos              1.179301
Formosa                 2.158276
Jujuy                   5.288225
La Pampa               -5.042536
La Rioja               14.126083
Mendoza                 2.397884
Misiones                6.189315
Neuquén                10.843467
Río Negro               9.483248
Salta                  -1.205953
San Juan                1.384838
San Luis                1.303652
Santa Cruz              0.027383
Santa Fe               -1.659965
Santiago del Estero    -0.262960
TdF                    10.346276
Tucumán                -0.065086
dtype: float64

In [ ]:
print((ratio_minutos_promedio_no_pago[['Varon', 'Mujer']].max()-ratio_minutos_promedio_no_pago[['Varon', 'Mujer']].min()).round(1))

In [14]:
print((ratio_minutos_promedio_no_pago[['Varon', 'Mujer']].max()-ratio_minutos_promedio_no_pago[['Varon', 'Mujer']].min()).round(1))

Varon    40.7
Mujer    38.2
dtype: float64


In [22]:
pd.concat([componente_insercion['Insercion laboral'], ratio_actividad['Mujer']], axis=1).sort_values(by='Mujer')

,Insercion laboral,Mujer
PROVINCIA,,
Formosa,80.093027,27.964823
Santiago del Estero,76.629318,39.009946
Chaco,80.592251,39.804082
San Juan,78.052350,41.392353
Corrientes,81.283223,42.856719
Tucumán,75.863304,45.242241
San Luis,78.206990,46.174859
Río Negro,83.689632,46.201577
Misiones,76.312885,46.387696


In [24]:
ratio_actividad['Actividad'].mean().round(1)

68.8